In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Bdu5WVAIe-vG5OgM8sq_y4zWJdRYq9KhOqN66NVP9RjM',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'predictionofenergyoutputofwindtur-donotdelete-pr-omqyaboy2mnabw'
object_key = 'Windfarm.xlsx'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']

df = pd.read_excel(body.read())
df.head()
Time(year-month-day h:m:s)	Wind speed at height of 10 meters (m/s)	Wind direction at height of 10 meters (˚)	Wind speed at height of 30 meters (m/s)	Wind direction at height of 30 meters (˚)	Wind speed at height of 50 meters (m/s)	Wind direction at height of 50 meters (˚)	Wind speed - at the height of wheel hub (m/s)	Wind speed - at the height of wheel hub (˚)	Air temperature (°C)	Atmosphere (hpa)	Relative humidity (%)	Power (MW)
0	2019-01-01 00:00:00	2.209	81.317	1.991	74.814	2.094	77.667	2.494	74.500	-13.484	889.867	76.320	0.254383
1	2019-01-01 00:15:00	1.828	77.460	1.698	75.048	1.757	88.733	1.882	74.367	-13.691	889.575	76.757	0.329703
2	2019-01-01 00:30:00	2.193	86.700	2.313	84.688	2.344	89.100	2.350	89.000	-13.766	889.942	76.981	0.296306
3	2019-01-01 00:45:00	2.654	78.160	2.494	74.939	2.574	87.267	2.808	82.733	-13.691	889.675	76.821	0.187590
4	2019-01-01 01:00:00	2.249	94.297	2.192	91.140	2.558	96.900	2.924	92.967	-13.447	890.025	74.571	0.081005
IMPORTING THE REQUIRED LIBRARIES
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedStratifiedKFold
from scipy.stats import loguniform
#Checking for null values
df.isnull().any()
Time(year-month-day h:m:s)                       False
Wind speed at height of 10 meters (m/s)          False
Wind direction at height of 10 meters (˚)        False
Wind speed at height of 30 meters (m/s)          False
Wind direction at height of 30 meters (˚)        False
Wind speed at height of 50 meters (m/s)          False
Wind direction at height of 50 meters (˚)        False
Wind speed - at the height of wheel hub (m/s)    False
Wind speed - at the height of wheel hub (˚)      False
Air temperature (°C)                             False
Atmosphere (hpa)                                 False
Relative humidity (%)                            False
Power (MW)                                       False
dtype: bool
#renaming a column name
df.rename(columns = {'Power (MW)':'Power'}, inplace = True)
df.rename(columns={'Time(year-month-day h:m:s)':'Time'},inplace = True)
df
Time	Wind speed at height of 10 meters (m/s)	Wind direction at height of 10 meters (˚)	Wind speed at height of 30 meters (m/s)	Wind direction at height of 30 meters (˚)	Wind speed at height of 50 meters (m/s)	Wind direction at height of 50 meters (˚)	Wind speed - at the height of wheel hub (m/s)	Wind speed - at the height of wheel hub (˚)	Air temperature (°C)	Atmosphere (hpa)	Relative humidity (%)	Power
0	2019-01-01 00:00:00	2.209	81.317	1.991	74.814	2.094	77.667	2.494	74.500	-13.484	889.867	76.320	0.254383
1	2019-01-01 00:15:00	1.828	77.460	1.698	75.048	1.757	88.733	1.882	74.367	-13.691	889.575	76.757	0.329703
2	2019-01-01 00:30:00	2.193	86.700	2.313	84.688	2.344	89.100	2.350	89.000	-13.766	889.942	76.981	0.296306
3	2019-01-01 00:45:00	2.654	78.160	2.494	74.939	2.574	87.267	2.808	82.733	-13.691	889.675	76.821	0.187590
4	2019-01-01 01:00:00	2.249	94.297	2.192	91.140	2.558	96.900	2.924	92.967	-13.447	890.025	74.571	0.081005
...	...	...	...	...	...	...	...	...	...	...	...	...	...
70171	2020-12-31 22:45:00	5.738	234.600	6.408	235.454	6.575	242.033	6.680	238.433	-9.108	890.008	53.312	13.633642
70172	2020-12-31 23:00:00	5.748	229.333	6.231	233.423	6.372	239.800	6.448	236.833	-9.978	889.817	53.877	6.869759
70173	2020-12-31 23:15:00	4.468	235.133	4.406	253.261	4.381	263.400	4.399	261.733	-9.808	889.825	54.437	4.352931
70174	2020-12-31 23:30:00	3.730	242.580	3.280	252.492	3.204	268.600	3.446	266.733	-9.633	889.492	54.677	7.185250
70175	2020-12-31 23:45:00	3.291	243.733	3.311	254.267	3.643	266.733	4.306	268.633	-8.991	889.492	53.381	12.384466
70176 rows × 13 columns

SPLITTING THE VARIABLES INTO DEPENDENT AND INDEPENDENT VARIABLES
#dependent variable
y=df['Power']
y
0         0.254383
1         0.329703
2         0.296306
3         0.187590
4         0.081005
           ...    
70171    13.633642
70172     6.869759
70173     4.352931
70174     7.185250
70175    12.384466
Name: Power, Length: 70176, dtype: float64
#independent variable
X=df.drop(columns=['Power','Time'],axis=1)
X.head()
Wind speed at height of 10 meters (m/s)	Wind direction at height of 10 meters (˚)	Wind speed at height of 30 meters (m/s)	Wind direction at height of 30 meters (˚)	Wind speed at height of 50 meters (m/s)	Wind direction at height of 50 meters (˚)	Wind speed - at the height of wheel hub (m/s)	Wind speed - at the height of wheel hub (˚)	Air temperature (°C)	Atmosphere (hpa)	Relative humidity (%)
0	2.209	81.317	1.991	74.814	2.094	77.667	2.494	74.500	-13.484	889.867	76.320
1	1.828	77.460	1.698	75.048	1.757	88.733	1.882	74.367	-13.691	889.575	76.757
2	2.193	86.700	2.313	84.688	2.344	89.100	2.350	89.000	-13.766	889.942	76.981
3	2.654	78.160	2.494	74.939	2.574	87.267	2.808	82.733	-13.691	889.675	76.821
4	2.249	94.297	2.192	91.140	2.558	96.900	2.924	92.967	-13.447	890.025	74.571
SPLITTING THE DATASET INTO TRAINING DATA AND TESTING DATA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)
#0.3 indicates 30% test dataset and remaining 70% training dataset which is ideal size of dataset for ml algorithms training and testing 
X_train
Wind speed at height of 10 meters (m/s)	Wind direction at height of 10 meters (˚)	Wind speed at height of 30 meters (m/s)	Wind direction at height of 30 meters (˚)	Wind speed at height of 50 meters (m/s)	Wind direction at height of 50 meters (˚)	Wind speed - at the height of wheel hub (m/s)	Wind speed - at the height of wheel hub (˚)	Air temperature (°C)	Atmosphere (hpa)	Relative humidity (%)
63536	1.631	169.873	1.523	137.859	0.861	142.867	1.412	140.867	9.637	890.017	24.992
21654	4.767	294.853	4.646	292.855	5.020	298.733	5.605	296.467	31.287	889.467	10.667
2093	1.602	259.433	1.888	282.533	3.430	316.200	5.276	325.333	-6.734	889.533	61.360
61223	13.250	288.053	14.901	282.997	14.910	286.033	15.507	284.533	8.494	889.675	58.448
31193	10.090	235.367	11.646	257.968	12.232	262.367	12.430	258.833	-3.739	889.925	60.640
...	...	...	...	...	...	...	...	...	...	...	...
21243	7.177	219.620	8.028	214.812	7.453	218.833	6.667	215.633	21.352	889.583	20.891
45891	1.383	180.110	0.060	89.039	0.506	92.133	1.857	86.000	11.978	889.633	11.776
42613	4.724	249.713	5.846	258.673	6.221	264.933	6.434	262.133	10.316	889.817	41.109
43567	12.263	244.300	13.435	275.286	14.030	277.833	14.614	275.733	2.877	889.792	60.389
68268	7.835	264.867	9.143	256.757	9.553	257.983	9.709	251.083	-11.953	889.867	58.155
49123 rows × 11 columns

# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
TRAINING THE MODEL
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor (n_estimators=30, random_state=0)
regressor.fit(X_train,y_train)
RandomForestRegressor(n_estimators=30, random_state=0)
TESTING THE MODEL
y_pred=regressor.predict(X_test)
y_pred
array([33.26070127, 25.18657937,  0.32804497, ...,  1.21720073,
        0.49453063, 70.8779573 ])
Acc=pd.DataFrame({'Actual_y_value':y_test,'Predicted_y_value':y_pred})
Acc
Actual_y_value	Predicted_y_value
50585	41.646313	33.260701
1610	26.752834	25.186579
68970	0.186879	0.328045
66444	1.792049	0.463834
38116	0.140692	0.483375
...	...	...
43785	7.865973	8.338973
34494	49.145634	51.113949
30848	1.079351	1.217201
33926	0.900288	0.494531
54458	84.859420	70.877957
21053 rows × 2 columns

EVALUATION USING METRICS
from sklearn.metrics import r2_score

r2_score(y_test, y_pred)
0.8849999426996336
IBM-DEPLOYMENT
!pip install -U ibm-watson-machine-learning
Requirement already satisfied: ibm-watson-machine-learning in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (1.0.257)
Requirement already satisfied: certifi in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from ibm-watson-machine-learning) (2022.9.24)
Requirement already satisfied: packaging in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from ibm-watson-machine-learning) (21.3)
Requirement already satisfied: requests in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from ibm-watson-machine-learning) (2.26.0)
Requirement already satisfied: lomond in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from ibm-watson-machine-learning) (0.3.3)
Requirement already satisfied: tabulate in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from ibm-watson-machine-learning) (0.8.9)
Requirement already satisfied: ibm-cos-sdk==2.11.* in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from ibm-watson-machine-learning) (2.11.0)
Requirement already satisfied: urllib3 in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from ibm-watson-machine-learning) (1.26.7)
Requirement already satisfied: pandas<1.5.0,>=0.24.2 in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from ibm-watson-machine-learning) (1.3.4)
Requirement already satisfied: importlib-metadata in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from ibm-watson-machine-learning) (4.8.2)
Requirement already satisfied: ibm-cos-sdk-s3transfer==2.11.0 in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from ibm-cos-sdk==2.11.*->ibm-watson-machine-learning) (2.11.0)
Requirement already satisfied: jmespath<1.0.0,>=0.7.1 in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from ibm-cos-sdk==2.11.*->ibm-watson-machine-learning) (0.10.0)
Requirement already satisfied: ibm-cos-sdk-core==2.11.0 in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from ibm-cos-sdk==2.11.*->ibm-watson-machine-learning) (2.11.0)
Requirement already satisfied: python-dateutil<3.0.0,>=2.1 in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from ibm-cos-sdk-core==2.11.0->ibm-cos-sdk==2.11.*->ibm-watson-machine-learning) (2.8.2)
Requirement already satisfied: pytz>=2017.3 in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from pandas<1.5.0,>=0.24.2->ibm-watson-machine-learning) (2021.3)
Requirement already satisfied: numpy>=1.17.3 in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from pandas<1.5.0,>=0.24.2->ibm-watson-machine-learning) (1.20.3)
Requirement already satisfied: six>=1.5 in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from python-dateutil<3.0.0,>=2.1->ibm-cos-sdk-core==2.11.0->ibm-cos-sdk==2.11.*->ibm-watson-machine-learning) (1.15.0)
Requirement already satisfied: idna<4,>=2.5 in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from requests->ibm-watson-machine-learning) (3.3)
Requirement already satisfied: charset-normalizer~=2.0.0 in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from requests->ibm-watson-machine-learning) (2.0.4)
Requirement already satisfied: zipp>=0.5 in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from importlib-metadata->ibm-watson-machine-learning) (3.6.0)
Requirement already satisfied: pyparsing!=3.0.5,>=2.0.2 in /opt/conda/envs/Python-3.9/lib/python3.9/site-packages (from packaging->ibm-watson-machine-learning) (3.0.4)
from ibm_watson_machine_learning import APIClient
import json
AUTHENTICATE AND SET SPACE
wml_credentials = {
    "apikey":"Ke3Pq9j_CPfm7SmFG5cJerK2XZ4J-AeEMDYrAlxIxnA1",
    "url":"https://us-south.ml.cloud.ibm.com"
}
wml_client = APIClient(wml_credentials)
wml_client.spaces.list()
Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ---------------  ------------------------
ID                                    NAME             CREATED
6df2e433-7c6e-4d6d-95ef-4abe297a223b  wind_prediction  2022-11-19T07:29:13.725Z
------------------------------------  ---------------  ------------------------
SPACE_ID= "6df2e433-7c6e-4d6d-95ef-4abe297a223b"
wml_client.set.default_space(SPACE_ID)
'SUCCESS'
wml_client.software_specifications.list(500)
-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10          125b6d9a-5b1f-5e8d-972a-b251688ccf40  base
runtime-22.1-py3.9               12b83a17-24d8-5082-900f-0ab31fbfd3cb  base
scikit-learn_0.22-py3.6          154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                     1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6           1bc6029a-cc97-56da-b8e0-39c3880dbbe7  base
kernel-spark3.3-r3.6             1c9e5454-f216-59dd-a20e-474a5cdf5988  base
pytorch-onnx_rt22.1-py3.9-edt    1d362186-7ad5-5b59-8b6c-9d0880bde37f  base
tensorflow_2.1-py3.6             1eb25b84-d6ed-5dde-b6a5-3fbdf1665666  base
spark-mllib_3.2                  20047f72-0a98-58c7-9ff5-a77b012eb8f5  base
tensorflow_2.4-py3.8-horovod     217c16f6-178f-56bf-824a-b19f20564c49  base
runtime-22.1-py3.9-cuda          26215f05-08c3-5a41-a1b0-da66306ce658  base
do_py3.8                         295addb5-9ef9-547e-9bf4-92ae3563e720  base
autoai-ts_3.8-py3.8              2aa0c932-798f-5ae9-abd6-15e0c2402fb5  base
tensorflow_1.15-py3.6            2b73a275-7cbf-420b-a912-eae7f436e0bc  base
kernel-spark3.3-py3.9            2b7961e2-e3b1-5a8c-a491-482c8368839a  base
pytorch_1.2-py3.6                2c8ef57d-2687-4b7d-acce-01f94976dac1  base
spark-mllib_2.3                  2e51f700-bca0-4b0d-88dc-5c6791338875  base
pytorch-onnx_1.1-py3.6-edt       32983cea-3f32-4400-8965-dde874a8d67e  base
spark-mllib_3.0-py37             36507ebe-8770-55ba-ab2a-eafe787600e9  base
spark-mllib_2.4                  390d21f8-e58b-4fac-9c55-d7ceda621326  base
autoai-ts_rt22.2-py3.10          396b2e83-0953-5b86-9a55-7ce1628a406f  base
xgboost_0.82-py3.6               39e31acd-5f30-41dc-ae44-60233c80306e  base
pytorch-onnx_1.2-py3.6-edt       40589d0e-7019-4e28-8daa-fb03b6f4fe12  base
pytorch-onnx_rt22.2-py3.10       40e73f55-783a-5535-b3fa-0c8b94291431  base
default_r36py38                  41c247d3-45f8-5a71-b065-8580229facf0  base
autoai-ts_rt22.1-py3.9           4269d26e-07ba-5d40-8f66-2d495b0c71f7  base
autoai-obm_3.0                   42b92e18-d9ab-567f-988a-4240ba1ed5f7  base
pmml-3.0_4.3                     493bcb95-16f1-5bc5-bee8-81b8af80e9c7  base
spark-mllib_2.4-r_3.6            49403dff-92e9-4c87-a3d7-a42d0021c095  base
xgboost_0.90-py3.6               4ff8d6c2-1343-4c18-85e1-689c965304d3  base
pytorch-onnx_1.1-py3.6           50f95b2a-bc16-43bb-bc94-b0bed208c60b  base
autoai-ts_3.9-py3.8              52c57136-80fa-572e-8728-a5e7cbb42cde  base
spark-mllib_2.4-scala_2.11       55a70f99-7320-4be5-9fb9-9edb5a443af5  base
spark-mllib_3.0                  5c1b0ca2-4977-5c2e-9439-ffd44ea8ffe9  base
autoai-obm_2.0                   5c2e37fa-80b8-5e77-840f-d912469614ee  base
spss-modeler_18.1                5c3cad7e-507f-4b2a-a9a3-ab53a21dee8b  base
cuda-py3.8                       5d3232bf-c86b-5df4-a2cd-7bb870a1cd4e  base
runtime-22.2-py3.10-xc           5e8cddff-db4a-5a6a-b8aa-2d4af9864dab  base
autoai-kb_3.1-py3.7              632d4b22-10aa-5180-88f0-f52dfb6444d7  base
pytorch-onnx_1.7-py3.8           634d3cdc-b562-5bf9-a2d4-ea90a478456b  base
spark-mllib_2.3-r_3.6            6586b9e3-ccd6-4f92-900f-0f8cb2bd6f0c  base
tensorflow_2.4-py3.7             65e171d7-72d1-55d9-8ebb-f813d620c9bb  base
spss-modeler_18.2                687eddc9-028a-4117-b9dd-e57b36f1efa5  base
pytorch-onnx_1.2-py3.6           692a6a4d-2c4d-45ff-a1ed-b167ee55469a  base
spark-mllib_2.3-scala_2.11       7963efe5-bbec-417e-92cf-0574e21b4e8d  base
spark-mllib_2.4-py37             7abc992b-b685-532b-a122-a396a3cdbaab  base
caffe_1.0-py3.6                  7bb3dbe2-da6e-4145-918d-b6d84aa93b6b  base
pytorch-onnx_1.7-py3.7           812c6631-42b7-5613-982b-02098e6c909c  base
cuda-py3.6                       82c79ece-4d12-40e6-8787-a7b9e0f62770  base
tensorflow_1.15-py3.6-horovod    8964680e-d5e4-5bb8-919b-8342c6c0dfd8  base
hybrid_0.1                       8c1a58c6-62b5-4dc4-987a-df751c2756b6  base
pytorch-onnx_1.3-py3.7           8d5d8a87-a912-54cf-81ec-3914adaa988d  base
caffe-ibm_1.0-py3.6              8d863266-7927-4d1e-97d7-56a7f4c0a19b  base
runtime-22.2-py3.10-cuda         8ef391e4-ef58-5d46-b078-a82c211c1058  base
spss-modeler_17.1                902d0051-84bd-4af6-ab6b-8f6aa6fdeabb  base
do_12.10                         9100fd72-8159-4eb9-8a0b-a87e12eefa36  base
do_py3.7                         9447fa8b-2051-4d24-9eef-5acb0e3c59f8  base
spark-mllib_3.0-r_3.6            94bb6052-c837-589d-83f1-f4142f219e32  base
cuda-py3.7-opence                94e9652b-7f2d-59d5-ba5a-23a414ea488f  base
nlp-py3.8                        96e60351-99d4-5a1c-9cc0-473ac1b5a864  base
cuda-py3.7                       9a44990c-1aa1-4c7d-baf8-c4099011741c  base
hybrid_0.2                       9b3f9040-9cee-4ead-8d7a-780600f542f7  base
spark-mllib_3.0-py38             9f7a8fc1-4d3c-5e65-ab90-41fa8de2d418  base
autoai-kb_3.3-py3.7              a545cca3-02df-5c61-9e88-998b09dc79af  base
spark-mllib_3.0-py39             a6082a27-5acc-5163-b02c-6b96916eb5e0  base
runtime-22.1-py3.9-do            a7e7dbf1-1d03-5544-994d-e5ec845ce99a  base
default_py3.8                    ab9e1b80-f2ce-592c-a7d2-4f2344f77194  base
tensorflow_rt22.1-py3.9          acd9c798-6974-5d2f-a657-ce06e986df4d  base
kernel-spark3.2-py3.9            ad7033ee-794e-58cf-812e-a95f4b64b207  base
autoai-obm_2.0 with Spark 3.0    af10f35f-69fa-5d66-9bf5-acb58434263a  base
runtime-22.2-py3.10              b56101f1-309d-549b-a849-eaa63f77b2fb  base
default_py3.7_opence             c2057dd4-f42c-5f77-a02f-72bdbd3282c9  base
tensorflow_2.1-py3.7             c4032338-2a40-500a-beef-b01ab2667e27  base
do_py3.7_opence                  cc8f8976-b74a-551a-bb66-6377f8d865b4  base
spark-mllib_3.3                  d11f2434-4fc7-58b7-8a62-755da64fdaf8  base
autoai-kb_3.0-py3.6              d139f196-e04b-5d8b-9140-9a10ca1fa91a  base
spark-mllib_3.0-py36             d82546d5-dd78-5fbb-9131-2ec309bc56ed  base
autoai-kb_3.4-py3.8              da9b39c3-758c-5a4f-9cfd-457dd4d8c395  base
kernel-spark3.2-r3.6             db2fe4d6-d641-5d05-9972-73c654c60e0a  base
autoai-kb_rt22.1-py3.9           db6afe93-665f-5910-b117-d879897404d9  base
tensorflow_rt22.1-py3.9-horovod  dda170cc-ca67-5da7-9b7a-cf84c6987fae  base
autoai-ts_1.0-py3.7              deef04f0-0c42-5147-9711-89f9904299db  base
tensorflow_2.1-py3.7-horovod     e384fce5-fdd1-53f8-bc71-11326c9c635f  base
default_py3.7                    e4429883-c883-42b6-87a8-f419d64088cd  base
do_22.1                          e51999ba-6452-5f1f-8287-17228b88b652  base
autoai-obm_3.2                   eae86aab-da30-5229-a6a6-1d0d4e368983  base
runtime-22.2-r4.2                ec0a3d28-08f7-556c-9674-ca7c2dba30bd  base
tensorflow_rt22.2-py3.10         f65bd165-f057-55de-b5cb-f97cf2c0f393  base
do_20.1                          f686cdd9-7904-5f9d-a732-01b0d6b10dc5  base
pytorch-onnx_rt22.2-py3.10-edt   f8a05d07-e7cd-57bb-a10b-23f1d4b837ac  base
scikit-learn_0.19-py3.6          f963fa9d-4bb7-5652-9c5d-8d9289ef6ad9  base
tensorflow_2.4-py3.8             fe185c44-9a99-5425-986b-59bd1d2eda46  base
-------------------------------  ------------------------------------  ----
SAVE AND DEPLOY THE MODEL
import sklearn
sklearn.__version__
'1.0.2'
MODEL_NAME = 'windpower'
DEPLOYMENT_NAME = 'wind_power_prediction'
DEMO_MODEL = regressor
# Set Python Version
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')
# Setup model meta
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}
#Save model
model_details = wml_client.repository.store_model(
    model=DEMO_MODEL, 
    meta_props=model_props, 
    training_data=X_train, 
    training_target=y_train
)
model_details
{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'Power',
  'schemas': {'input': [{'fields': [{'name': 'f0', 'type': 'float'},
      {'name': 'f1', 'type': 'float'},
      {'name': 'f2', 'type': 'float'},
      {'name': 'f3', 'type': 'float'},
      {'name': 'f4', 'type': 'float'},
      {'name': 'f5', 'type': 'float'},
      {'name': 'f6', 'type': 'float'},
      {'name': 'f7', 'type': 'float'},
      {'name': 'f8', 'type': 'float'},
      {'name': 'f9', 'type': 'float'},
      {'name': 'f10', 'type': 'float'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'metadata': {'created_at': '2022-11-19T12:35:21.663Z',
  'id': 'a9f1123c-0401-4b0f-9025-bcc1fc1b8c45',
  'modified_at': '2022-11-19T12:35:47.039Z',
  'name': 'windpower',
  'owner': 'IBMid-6670006K3L',
  'resource_key': '0cd17da7-1387-411e-b620-d57c3d739d72',
  'space_id': '6df2e433-7c6e-4d6d-95ef-4abe297a223b'},
 'system': {'warnings': []}}
model_id = wml_client.repository.get_model_id(model_details)
model_id
'a9f1123c-0401-4b0f-9025-bcc1fc1b8c45'
# Set meta
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}
# Deploy
deployment = wml_client.deployments.create(
    artifact_uid=model_id, 
    meta_props=deployment_props 
)
#######################################################################################

Synchronous deployment creation for uid: 'a9f1123c-0401-4b0f-9025-bcc1fc1b8c45' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='81e352a8-8cbb-41f3-8518-fae3177d7c89'
------------------------------------------------------------------------------------------------


 